# Lab 8: Data Loading Simulation
This notebook simulates **three data loading methods** into DuckDB:

- ✅ Append (no duplicates)
- ✅ Truncate and reload
- ✅ Incremental (insert or update)

It loads CPI data from the Philadelphia Fed API and tests all methods over a simulated daily range.

In [3]:
!pip3 install duckdb pandas openpyxl requests

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [10]:
import duckdb
import openpyxl
import requests
import load_inc
import pandas as pd
from datetime import datetime, timedelta
import time
import os
import load_append
import load_trunc

In [11]:
# === Simulation Configuration ===
start_date = datetime(2004, 1, 1)
end_date = datetime(2004, 3, 1)  # You can change this for longer tests

# Initialize timing lists
append_times = []
trunc_times = []
inc_times = []

In [6]:
# === Run Loaders Over Date Range ===
current_date = start_date
while current_date <= end_date:
    pull_date_str = current_date.strftime("%Y-%m-%d")
    print(f"\n📅 Running loaders for: {pull_date_str}")

    try:
        t0 = time.time()
        load_append.run(pull_date_str)
        append_times.append(time.time() - t0)
    except Exception as e:
        print(f"[append error] {pull_date_str}: {e}")

    try:
        t0 = time.time()
        load_trunc.run(pull_date_str)
        trunc_times.append(time.time() - t0)
    except Exception as e:
        print(f"[trunc error] {pull_date_str}: {e}")

    try:
        t0 = time.time()
        load_inc.run(pull_date_str)
        inc_times.append(time.time() - t0)
    except Exception as e:
        print(f"[incremental error] {pull_date_str}: {e}")

    current_date += timedelta(days=1)



📅 Running loaders for: 2004-01-01
[append] Pull date 2004-01-01: Inserted 684 new rows.
[truncate] Pull date 2004-01-01: Replaced with 684 rows.
[incremental] Pull date 2004-01-01: Inserted 684 new rows.

📅 Running loaders for: 2004-01-02
[append] Pull date 2004-01-02: No new data to insert (all dates already exist).
[truncate] Pull date 2004-01-02: Replaced with 684 rows.
[incremental] Pull date 2004-01-02: Updated 684 existing rows.

📅 Running loaders for: 2004-01-03
[append] Pull date 2004-01-03: No new data to insert (all dates already exist).
[truncate] Pull date 2004-01-03: Replaced with 684 rows.
[incremental] Pull date 2004-01-03: Updated 684 existing rows.

📅 Running loaders for: 2004-01-04
[append] Pull date 2004-01-04: No new data to insert (all dates already exist).
[truncate] Pull date 2004-01-04: Replaced with 684 rows.
[incremental] Pull date 2004-01-04: Updated 684 existing rows.

📅 Running loaders for: 2004-01-05
[append] Pull date 2004-01-05: No new data to insert (a

In [7]:
# === Preview Data from Each Table ===
def view_table(db_path, table_name):
    con = duckdb.connect(db_path)
    df = con.execute(f"SELECT * FROM {table_name} ORDER BY date LIMIT 10").fetchdf()
    con.close()
    return df

print("\n📊 Sample from Append Table:")
display(view_table("economic_data_append.duckdb", "economic_data_append"))

print("\n📊 Sample from Trunc Table:")
display(view_table("economic_data_trunc.duckdb", "economic_data_trunc"))

print("\n📊 Sample from Incremental Table:")
display(view_table("economic_data_inc.duckdb", "economic_data_inc"))



📊 Sample from Append Table:


,date,cpi
0,1947-01-01,21.5
1,1947-02-01,21.6
2,1947-03-01,22.0
3,1947-04-01,22.0
4,1947-05-01,22.0
5,1947-06-01,22.1
6,1947-07-01,22.2
7,1947-08-01,22.4
8,1947-09-01,22.8
9,1947-10-01,22.9



📊 Sample from Trunc Table:


,date,cpi
0,1947-01-01,21.5
1,1947-02-01,21.6
2,1947-03-01,22.0
3,1947-04-01,22.0
4,1947-05-01,22.0
5,1947-06-01,22.1
6,1947-07-01,22.2
7,1947-08-01,22.4
8,1947-09-01,22.8
9,1947-10-01,22.9



📊 Sample from Incremental Table:


,date,cpi
0,1947-01-01,21.5
1,1947-02-01,21.6
2,1947-03-01,22.0
3,1947-04-01,22.0
4,1947-05-01,22.0
5,1947-06-01,22.1
6,1947-07-01,22.2
7,1947-08-01,22.4
8,1947-09-01,22.8
9,1947-10-01,22.9


In [8]:
# === Compare Method Performance ===
def summarize_times(label, times):
    avg = round(sum(times) / len(times), 4)
    return f"{label} avg: {avg:.4f}s over {len(times)} runs"

print("\n⏱️ Performance Summary:")
print(summarize_times("Append", append_times))
print(summarize_times("Trunc", trunc_times))
print(summarize_times("Incremental", inc_times))



⏱️ Performance Summary:
Append avg: 1.8879s over 61 runs
Trunc avg: 1.8716s over 61 runs
Incremental avg: 1.9536s over 61 runs


In [9]:
# === Check for Duplicates in Append Table ===
con = duckdb.connect("economic_data_append.duckdb")
dup_check = con.execute("""
    SELECT date, COUNT(*) as count
    FROM economic_data_append
    GROUP BY date
    HAVING COUNT(*) > 1
""").fetchdf()
con.close()

if dup_check.empty:
    print("\n✅ No duplicates in append table.")
else:
    print("\n⚠️ Duplicates found in append table:")
    display(dup_check)



✅ No duplicates in append table.
